In [1]:
import os
import h5py
import numpy as np

save_path = "/media/temp/DATA21/Project_FAST/CODE/MIL_CLIP_Adapter/output_CAMELYON_feat_224x224_5x_CLIP(RN50)"
for split in ['train']:
    all_patches = np.array(h5py.File(os.path.join(save_path, f"{split}_patch_feat.h5"), 'r')['dataset_1'])
    patch_corresponding_slide_label = np.load(os.path.join(save_path, f"{split}_patch_corresponding_slide_label.npy"))
    patch_corresponding_slide_index = np.load(os.path.join(save_path, f"{split}_patch_corresponding_slide_index.npy"))
    patch_corresponding_slide_name = np.load(os.path.join(save_path, f"{split}_patch_corresponding_slide_name.npy"))
    patch_label = np.load(os.path.join(save_path, f"{split}_patch_label.npy"))

    print(split)
    print(f'{split}_all_patches.shape:', all_patches.shape)
    print('\tNaN:', np.isnan(all_patches).sum())
    print(f'{split}_patch_corresponding_slide_label.shape:', patch_corresponding_slide_label.shape)
    print(f'{split}_patch_corresponding_slide_index.shape:', patch_corresponding_slide_index.shape)
    print(f'{split}_patch_corresponding_slide_name.shape:', patch_corresponding_slide_name.shape)
    print(f'{split}_patch_label.shape:', patch_label.shape)
    print()

    num_patches = all_patches.shape[0]
    num_slides = patch_corresponding_slide_index.max() + 1

    # 2. sort instances features into bag
    slide_feat_all = []
    slide_label_all = []
    slide_patch_label_all = []
    for i in range(num_slides):
        idx_from_same_slide = patch_corresponding_slide_index == i
        # print(idx_from_same_slide)
        idx_from_same_slide = np.nonzero(idx_from_same_slide)[0]
        # print(idx_from_same_slide)

        slide_feat_all.append(all_patches[idx_from_same_slide])
        if (patch_corresponding_slide_label[idx_from_same_slide].max() != 
            patch_corresponding_slide_label[idx_from_same_slide].min()):
            raise
        slide_label_all.append(patch_corresponding_slide_label[idx_from_same_slide].max())
        slide_patch_label_all.append(patch_label[idx_from_same_slide])
    print(f"[DATA INFO] num_slide is {num_slides}; num_patches is {num_patches}")
    print(f"[DATA INFO] #pos_slide is {sum(slide_label_all)}, #neg_slide is {len(slide_label_all) - sum(slide_label_all)}")

train
train_all_patches.shape: (568842, 1024)
	NaN: 0
train_patch_corresponding_slide_label.shape: (568842,)
train_patch_corresponding_slide_index.shape: (568842,)
train_patch_corresponding_slide_name.shape: (568842,)
train_patch_label.shape: (568842,)

[DATA INFO] num_slide is 251; num_patches is 568842
[DATA INFO] #pos_slide is 92, #neg_slide is 159


In [15]:
import pandas as pd

# # Set display options to show all rows
# pd.set_option('display.max_rows', None)

# Create data for each slide
data = {
    'slide_id': range(len(slide_label_all)),
    'slide_label': slide_label_all,
    'pos_instances': [l.sum() for l in slide_patch_label_all],
    'total_instances': [len(l) for l in slide_patch_label_all]
}

# Create dataframe
df = pd.DataFrame(data)

# Calculate positive ratio
df['pos_ratio'] = df['pos_instances'] / df['total_instances']

# Sort by number of positive instances
df = df.sort_values('pos_instances', ascending=False)

display(df)

# Reset display options back to default
pd.reset_option('display.max_rows')

,slide_id,slide_label,pos_instances,total_instances,pos_ratio
25,25,1,2908,4015,0.724284
58,58,1,2163,5776,0.374481
65,65,1,1683,3266,0.515309
91,91,1,1312,2548,0.514914
84,84,1,1312,2525,0.519604
...,...,...,...,...,...
110,110,0,0,3142,0.000000
112,112,0,0,2379,0.000000
113,113,0,0,1339,0.000000
115,115,0,0,1584,0.000000


In [18]:
# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Split into positive and negative dataframes
pos_df = df[df['slide_label'] == 1].sort_values('pos_instances', ascending=False).reset_index(drop=True)
neg_df = df[df['slide_label'] == 0].sort_values('total_instances', ascending=False).reset_index(drop=True)

# Display positive slides
print("Positive Slides (sorted by number of positive instances):")
display(pos_df)

print("\nNegative Slides (sorted by total number of instances):")
display(neg_df)

# # Print summary statistics
# print(f"\nSummary:")
# print(f"Number of positive slides: {len(pos_df)}")
# print(f"Number of negative slides: {len(neg_df)}")
# print(f"Total number of slides: {len(df)}")

Positive Slides (sorted by number of positive instances):


,slide_id,slide_label,pos_instances,total_instances,pos_ratio
0,25,1,2908,4015,0.724284
1,58,1,2163,5776,0.374481
2,65,1,1683,3266,0.515309
3,91,1,1312,2548,0.514914
4,84,1,1312,2525,0.519604
5,169,1,1207,3355,0.359762
6,5,1,1069,5383,0.198588
7,34,1,1061,3373,0.314557
8,86,1,1055,2820,0.374113
9,96,1,948,4477,0.211749



Negative Slides (sorted by total number of instances):


,slide_id,slide_label,pos_instances,total_instances,pos_ratio
0,243,0,0,6756,0.0
1,214,0,0,6615,0.0
2,135,0,0,6563,0.0
3,33,0,0,5618,0.0
4,181,0,0,5609,0.0
5,229,0,0,5134,0.0
6,117,0,0,4934,0.0
7,71,0,0,4792,0.0
8,134,0,0,4768,0.0
9,186,0,0,4422,0.0


In [19]:
pos_df.to_csv('camelyon_pos_slides.csv', index=False)
neg_df.to_csv('camelyon_neg_slides.csv', index=False)